In [ ]:
import numpy as np
import igl
import meshplot as mp
from scipy.spatial.transform import Rotation
import ipywidgets as iw
import time
import scipy.sparse
import scipy.sparse.linalg


In [ ]:
v, f = igl.read_triangle_mesh('data/hand.off')
labels = np.load('data/hand.label.npy').astype(int)
v -= v.min(axis=0)
v /= v.max()
mp.plot(v, f, c=labels)

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.5, 0.19…

In [ ]:
# labels = np.load("data/woody-hi.label.npy")
# v, f = igl.read_triangle_mesh("data/woody-hi.off")
# p = mp.plot(v, f, c=labels)
# v -= v.min(axis=0)
# v /= v.max()

In [ ]:
# COMPUTING B: 
handles_indices = np.where(labels > 0)[0]
free_indices = np.where(labels == 0)[0]

L = igl.cotmatrix(v, f)
M = igl.massmatrix(v, f, igl.MASSMATRIX_TYPE_VORONOI)
Minv_diag = 1 / M.diagonal()
Minv = np.diag(Minv_diag)

# bi-Laplacian
Q2 = Minv @ L @ L
Q2_free = Q2[free_indices, :][:, free_indices]
Q2_mixed = Q2[free_indices, :][:, handles_indices]
rhs = -Q2_mixed @ v[handles_indices]

v_smooth = v.copy()
for i in range(3): #x,y,z
    v_smooth[free_indices, i]    = np.linalg.solve(Q2_free, rhs[:, i])
    v_smooth[handles_indices, i] = v[handles_indices, i]

B = v_smooth.copy()
mp.plot(v_smooth, f, c=labels)

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.5, 0.19…

In [ ]:
# COMPUTING B':
def position_deformer(target_pos):
    '''Fill in this function to change positions'''
    Bc = target_pos.copy()
    free = free_indices
    handles = handles_indices
    
    rhs = -Q2_mixed @ Bc[handles]

    for i in range(3): #x,y,z
        Bc[free, i] = np.linalg.solve(Q2_free, rhs[:, i])
        Bc[handles, i] = target_pos[handles_indices, i]

    return Bc
''' (Optional) Register this function to perform interactive deformation
pos_f.deformer = position_deformer
'''
Bc = position_deformer(B)
mp.plot(Bc, f, c=labels)
# pos_f.deformer = position_deformer

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.5, 0.19…

In [ ]:
handle_vertex_positions = B.copy()
# handle_vertex_positions = v.copy()

pos_f_saver = np.zeros((labels.max() + 1, 6))
def pos_f(s,x,y,z, α, β, γ):
    slices = (labels==s)
    r = Rotation.from_euler('xyz', [α, β, γ], degrees=True)
    v_slice = B[slices] + np.array([[x,y,z]])
    center = v_slice.mean(axis=0)
    handle_vertex_positions[slices] = r.apply(v_slice - center) + center
    pos_f_saver[s - 1] = [x,y,z,α,β,γ]
    t0 = time.time()
    v_deformed = pos_f.deformer(handle_vertex_positions)
    p.update_object(vertices = v_deformed)
    t1 = time.time()
    print("v.shape:", v.shape)
    print("labels.shape:", labels.shape)
    print("slices shape:", slices.shape)
    print("np.sum(slices):", np.sum(slices))  # How many vertices are selected

    print('FPS', 1/(t1 - t0))
# pos_f.deformer = lambda x:x
pos_f.deformer = position_deformer

In [ ]:
def widgets_wrapper():
    segment_widget = iw.Dropdown(options=np.arange(labels.max()) + 1)
    translate_widget = {i:iw.FloatSlider(min=-1, max=1, value=0) 
                        for i in 'xyz'}
    rotate_widget = {a:iw.FloatSlider(min=-90, max=90, value=0, step=1) 
                     for a in 'αβγ'}

    def update_seg(*args):
        (translate_widget['x'].value,translate_widget['y'].value,
        translate_widget['z'].value,
        rotate_widget['α'].value,rotate_widget['β'].value,
        rotate_widget['γ'].value) = pos_f_saver[segment_widget.value]
    segment_widget.observe(update_seg, 'value')
    widgets_dict = dict(s=segment_widget)
    widgets_dict.update(translate_widget)
    widgets_dict.update(rotate_widget)
    return widgets_dict

In [ ]:
## Widget UI

p = mp.plot(handle_vertex_positions, f, c=labels)
iw.interact(pos_f,
            **widgets_wrapper())

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.5, 0.19…

interactive(children=(Dropdown(description='s', options=(1, 2, 3, 4), value=1), FloatSlider(value=0.0, descrip…

<function __main__.pos_f(s, x, y, z, α, β, γ)>

In [ ]:
segments_recording = np.load("data/woody-hi.label.npy")
v, f = igl.read_triangle_mesh("data/woody-hi.off")
p = mp.plot(v, f, c=segments_recording)


Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(174.5, 20…